In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [2]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [3]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance31.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"), header=1, delim=",") |> DataFrame 
SDD = CSV.File(string(path,"SDD31.csv"),header=1,delim=",") |> DataFrame;

In [4]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)                          # 2021 global GD ($/yr)
es_ratio = global_sink/global_emi

emission_c = SDD[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = SDD[!, "sink ton/yr"]           # national CO2 sink (ton/yr)
Dsoc = SDD[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"];

In [5]:
cell_demand = 0.001*164.98*(1.369*1e6)*2;           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 164.98 kg/pack (35kwh/pack), 80~100 kWh per EV)

In [6]:
cell_demand = 500000;

In [52]:
function pct_count(x,y)
    proD = x .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink


    transD = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            emi = 0
            for j in 1:ncty
                emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  
            end
            transD[i,k] = emi
        end
    end

    MEO = proD - proS + transD;
    
    indices = (MEO .>= 0.1) .|| (MEO .<= -0.1)
    ls = MEO[indices]
    total = length(ls)
    
    num_os = 0
    num_safe = 0
    for e in ls
        if e > 0
            num_os += 1
        elseif e < 0
            num_safe += 1
        end
    end
    
    return MEO, num_os,num_safe

#     return MEO,num_os,num_safe,total
end

pct_count (generic function with 1 method)

In [278]:
function overshoot_pct()
    model = Model(Gurobi.Optimizer)
    set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)
    set_optimizer_attribute(model, MOI.Silent(), true)
    
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, 0<=obj<=1)

    @variable(model, delta[1:ncty, 1:nproc], Bin)
    @variable(model, z[1:ncty, 1:nproc], Bin)
    @variable(model, r[1:ncty, 1:nproc], Bin)
    @variable(model, t[1:ncty, 1:nproc], Bin);
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) >= cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    proD = x .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink

    transD = Matrix{AffExpr}(undef, ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            emi = 0
            for j in 1:ncty
                emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
            end
            transD[i,k] = emi
        end
    end
    
    slack = 1e-6
    M = 1e7
    
    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty];
    
    EO = proD - proS + transD
    for i in 1:ncty
        for k in 1:nproc
            @constraint(model, EO[i,k] >= -M*r[i,k] + slack*t[i,k])
            @constraint(model, EO[i,k] <= -slack*r[i,k] + M*t[i,k])
            @constraint(model, z[i,k] + r[i,k] + t[i,k] == 1)
            @constraint(model, delta[i,k] => {x[i,k] >= 0.1})
        end
    end
    
    @constraint(model, obj*sum(delta) == sum(t))
    @objective(model, Min, obj)
    JuMP.optimize!(model)
    
    xx = JuMP.value.(x)
    yy = JuMP.value.(y)
    zz = JuMP.value.(z)
    rr = JuMP.value.(r)
    tt = JuMP.value.(t)
    solution = JuMP.objective_value(model)
    
    return Dict(["x"=>xx, "obj"=>solution, "y"=>yy, "z"=>zz, "t"=>tt, "r"=>rr])
    
end

overshoot_pct (generic function with 1 method)

----

In [259]:
model = Model(Gurobi.Optimizer)
set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)

Academic license - for non-commercial use only - expires 2024-12-26


In [260]:
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, 0<=obj<=1)

@variable(model, delta[1:ncty, 1:nproc], Bin)
@variable(model, z[1:ncty, 1:nproc], Bin)
@variable(model, r[1:ncty, 1:nproc], Bin)
@variable(model, t[1:ncty, 1:nproc], Bin);

In [261]:
x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= x[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) >= cell_demand)
tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
for j in tmp
    @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
end

In [262]:
proD = x .* Matrix(regional_EF[:,2:end])
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = x .* pro_sink

transD = Matrix{AffExpr}(undef, ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        emi = 0
        for j in 1:ncty
            emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
        end
        transD[i,k] = emi
    end
end

In [263]:
slack = 1e-6

1.0e-6

In [264]:
M = 1e7

1.0e7

In [265]:
Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty];

In [266]:
EO = proD - proS + transD;

In [267]:
# for i in 1:ncty
#     for k in 1:nproc
#         @constraint(model, EO[i,k] >= -M*(t[i,k]-1) + slack)
#         @constraint(model, EO[i,k] <= M*t[i,k])
#         @constraint(model, delta[i,k] => {x[i,k] >= 0.1})
#     end
# end

In [268]:
for i in 1:ncty
    for k in 1:nproc
        @constraint(model, EO[i,k] >= -M*r[i,k] + slack*t[i,k])
        @constraint(model, EO[i,k] <= -slack*r[i,k] + M*t[i,k])
        @constraint(model, z[i,k] + r[i,k] + t[i,k] == 1)
        @constraint(model, delta[i,k] => {x[i,k] >= 0.1})
    end
end

In [269]:
# @constraint(model, obj*(sum(r) + sum(t)) == sum(t))
@constraint(model, obj*sum(delta) == sum(t))
@objective(model, Min, obj)

obj

In [270]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2835 rows, 16741 columns and 77562 nonzeros
Model fingerprint: 0x371fab0d
Model has 1 quadratic constraint
Model has 465 general constraints
Variable types: 14881 continuous, 1860 integer (1860 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+07]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-06, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2101 rows and 13632 columns
Presolve time: 0.08s
Presolved: 1269 rows, 3287 columns, 10346 nonzeros
Variable types: 2606 continuous, 681 integer (681 binary)

Root relaxation: objective 2.562444e-02, 1136 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Objective Bounds   

In [271]:
x = JuMP.value.(x)
y = JuMP.value.(y)
z = JuMP.value.(z)
r = JuMP.value.(r)
t = JuMP.value.(t)
delta = JuMP.value.(delta)
solution = JuMP.objective_value(model)

0.1702128085106383

In [276]:
sum(t), sum(r), sum(z), sum(delta)

(8.00000199985714, 320.0, 136.99999800014285, 47.0)

In [273]:
meo, nos, nsf = pct_count(x,y);

In [274]:
maximum(meo)

2.039760546779608e6

In [275]:
nos, nsf, nos/(nos+nsf) 

(9, 29, 0.23684210526315788)

In [252]:
meo

31×15 Matrix{Float64}:
       0.0            0.0        …  0.0         0.0          0.0
   82051.7            0.0           0.0         0.0          0.0
       0.0            0.0           0.0         0.0          0.0
      -0.118858  -65557.2           0.0        -4.36557e-11  0.0
 -103967.0            0.0           0.0         0.0          0.0
       0.0            0.0        …  0.0         0.0          0.0
       0.0            0.0           0.0         0.0          0.0
       0.0            9.18613e5     0.0         0.0          0.0
       0.0       -39785.9           0.0        -1.00001e-6   0.0
       0.0            0.0           0.0         0.0          0.0
       0.0            0.0        …  1.94419e5   0.0          0.0
       0.0            0.0           0.0         0.0          0.0
       0.0            0.0           0.0         0.0          0.0
       ⋮                         ⋱                           
       0.0            0.0           0.0         8.67362e-19  0.0
     

In [253]:
x

31×15 Matrix{Float64}:
     0.0       0.0       0.0       0.0  0.0  …      0.0  0.0             0.0
 61000.0       0.0       0.0       0.0  0.0         0.0  0.0             0.0
     0.0       0.0  881547.0       0.0  0.0         0.0  0.0             0.0
     0.1    3100.0       0.0  180000.0  0.0         0.0  1.0e6           0.0
 39000.0       0.0       0.0       0.0  0.0         0.0  0.1             0.0
     0.0       0.0       0.0       0.0  0.0  …      0.0  0.0             0.0
     0.0       0.0       0.0  110000.0  0.0         0.0  0.0             0.0
     0.0  145000.0       0.0       0.0  0.0         0.0  0.0             0.0
     0.0    1084.0       0.0   42000.0  0.0         0.0  3.10324e5       0.0
     0.0       0.0       0.0       0.0  0.0         0.0  0.0             0.0
     0.0       0.0       0.0       0.0  0.0  …  10000.0  0.0             0.0
     0.0       0.0       0.0       0.0  0.0         0.0  0.0             0.0
     0.0       0.0       0.0       0.0  0.0         0

In [254]:
for i in 1:ncty
    if x[i,1] > 1
        println(x[i,1])
    end
end

61000.0
39000.0


In [255]:
sum(delta)

46.0

In [256]:
tot = 0
for k in 1: nproc
    num = 0
    for i in 1:ncty
        if x[i,k] > 1
            num += 1
            tot += 1
        end
    end
    println(num)
end


2
4
2
4
1
5
2
3
3
3
1
3
1
2
1


In [257]:
tot

37

In [159]:
for k in 1:nproc
    println(sum(delta[:,k]))
end

3.0
4.0
2.0
4.0
1.0
5.0
4.0
4.0
4.0
3.0
1.0
4.0
1.0
5.0
1.0


In [28]:
# for i in 1:ncty
#     for k in 1:nproc
#         if MEO[i,k] > 0.1 || MEO[i,k] < -0.1
#             MEO[i,k] = 0
#         else
#             MEO[i,k] = 1
#         end
#     end
# end

----

In [128]:
model = Model(Gurobi.Optimizer)
set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)

@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, 0<=obj<=1)

@variable(model, z[1:ncty, 1:nproc], Bin)
@variable(model, r[1:ncty, 1:nproc], Bin)
@variable(model, t[1:ncty, 1:nproc], Bin);

Academic license - for non-commercial use only - expires 2024-12-26


In [129]:
x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= x[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) >= cell_demand)
tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
for j in tmp
    @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
end

In [130]:
proD = x .* Matrix(regional_EF[:,2:end])
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = x .* pro_sink

transD = Matrix{AffExpr}(undef, ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        emi = 0
        for j in 1:ncty
            emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
        end
        transD[i,k] = emi
    end
end

In [131]:
slack = 1e-4

0.0001

In [132]:
Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty];

In [133]:
EO = proD - proS + transD;

In [134]:
for i in 1:ncty
    for k in 1:nproc
        @constraint(model, z[i,k] => {EO[i,k] == 0})
        @constraint(model, r[i,k] => {EO[i,k] + slack <= 0})
        @constraint(model, t[i,k] => {EO[i,k] - slack >= 0})
        @constraint(model, z[i,k] + r[i,k] + t[i,k] == 1)
    end
end

In [135]:
@constraint(model, obj*(sum(r) + sum(t)) == sum(t))
@objective(model, Min, obj)

obj

In [ ]:
JuMP.optimize!(model)

In [127]:
JuMP.value.(z)

31×15 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮

In [116]:
model = Model(Gurobi.Optimizer)
@variable(model, x)
@variable(model, y)
@variable(model, z, Bin)
@constraint(model, z => {x + y <= 2})


Academic license - for non-commercial use only - expires 2024-12-26


z => {x + y <= 2.0}